In [1]:
import keras

2024-03-12 12:57:21.201096: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 12:57:21.201168: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-12 12:57:21.236182: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-12 12:57:21.311571: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-12 12:57:22.322982: W tensorflow/compiler/tf2

In [151]:
inputs = keras.Input((224, 224, 3))
model1 = keras.layers.TFSMLayer('../models/ViT/vit_b16_patch16_224-i1k_pretrained', call_endpoint="serving_default")
outputs = model1(inputs, training=False)
model1 = keras.Model(inputs, outputs)
model1.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ input_layer_8 (InputLayer)      │ (None, 224, 224, 3)       │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ tfsm_layer_2 (TFSMLayer)        │ [(None, 1000), (None, 12, │ 86,567,656 │
│                                 │ None, None), (None, 12,   │            │
│                                 │ None, None), (None, 12,   │            │
│                                 │ None, None), (None, 12,   │            │
│                                 │ None, None), (None, 12,   │            │
│                                 │ None, None), (None, 12,   │            │
│                                 │ None, None), (None, 12,   │            │
│                                 │ None, None), (None, 12,   │            │
│                                 │ None, None), (None, 12,   │            │
│                                 │ None, None), (None, 12,   │            │
│                                 │ None, None), (None, 12,   │            │
│                                 │ None, None), (None, 12,   │            │
│                                 │ None, None)]              │            │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 86,567,656 (330.23 MB)

 Trainable params: 86,567,656 (330.23 MB)

 Non-trainable params: 0 (0.00 B)

In [149]:
import numpy as np
test = np.random.rand(1, 224, 224, 3)

In [150]:
pred = model1.predict(test)

NameError: name 'model1' is not defined

In [8]:
pred.keys()

dict_keys(['output_1', 'output_2_transformer_block_0_att', 'output_2_transformer_block_10_att', 'output_2_transformer_block_11_att', 'output_2_transformer_block_1_att', 'output_2_transformer_block_2_att', 'output_2_transformer_block_3_att', 'output_2_transformer_block_4_att', 'output_2_transformer_block_5_att', 'output_2_transformer_block_6_att', 'output_2_transformer_block_7_att', 'output_2_transformer_block_8_att', 'output_2_transformer_block_9_att'])

In [14]:
pred['output_2_transformer_block_1_att'].shape

(1, 12, 197, 197)

In [123]:
from transformers import TFViTModel, create_optimizer, AutoImageProcessor
import tensorflow as tf

In [124]:
image_processor = AutoImageProcessor.from_pretrained("google/vit-base-patch32-224-in21k")
# load pre-trained ViT model
vit_model = TFViTModel.from_pretrained(
    "google/vit-base-patch32-224-in21k"
)
vit_model

All PyTorch model weights were used when initializing TFViTModel.

All the weights of TFViTModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFViTModel for predictions without further training.


In [131]:
inputs = image_processor(np.ones((1,3,224,224,)), return_tensors="tf")
logits = vit_model(**inputs)
logits['last_hidden_state']

<tf.Tensor: shape=(1, 50, 768), dtype=float32, numpy=
array([[[-0.04642715,  0.07819472,  0.10297327, ..., -0.00410401,
          0.07026549, -0.08159713],
        [-0.01770746, -0.04575405, -0.01447336, ...,  0.00891134,
         -0.03938118,  0.00908835],
        [-0.01938407, -0.04436754, -0.01690471, ...,  0.00544327,
         -0.04128317,  0.00743156],
        ...,
        [ 0.03982006,  0.16118445, -0.205389  , ..., -0.02016526,
          0.01014593, -0.13618861],
        [ 0.03145476,  0.17934817, -0.23624192, ..., -0.03867835,
          0.00920017, -0.15700538],
        [-0.11332527,  0.10698851, -0.01375464, ..., -0.07873282,
          0.02806762, -0.0242648 ]]], dtype=float32)>

In [132]:
vit_model.layers[0](**inputs)['last_hidden_state'].shape

TensorShape([1, 50, 768])

In [134]:
vit_model.vit, vit_model.layers

(<transformers.models.vit.modeling_tf_vit.TFViTMainLayer at 0x7e9be0a61e10>,
 [<transformers.models.vit.modeling_tf_vit.TFViTMainLayer at 0x7e9be0a61e10>])

In [135]:
vit_model.vit(np.ones((1,3,224,224,)))[0]

<tf.Tensor: shape=(1, 50, 768), dtype=float32, numpy=
array([[[ 0.17067873, -0.05104578, -0.04005868, ...,  0.00516569,
          0.02375122, -0.0439344 ],
        [-0.01032811, -0.0427994 , -0.01694578, ...,  0.01821133,
         -0.03151407,  0.00783023],
        [-0.01213216, -0.04085394, -0.018418  , ...,  0.01636786,
         -0.03094222,  0.00621234],
        ...,
        [ 0.12138271, -0.06599617, -0.16070645, ..., -0.02530573,
          0.01102774,  0.05817366],
        [ 0.07293798, -0.09610921, -0.21211904, ...,  0.02409113,
          0.04105248,  0.03535001],
        [ 0.01125343, -0.05172333, -0.26962224, ...,  0.17057098,
          0.09815656,  0.03744992]]], dtype=float32)>

In [136]:
dir(vit_model)

['_SCALAR_UPRANKING_ON',
 '_TF_MODULE_IGNORED_PROPERTIES',
 '__annotations__',
 '__call__',
 '__class__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activity_regularizer',
 '_add_trackable',
 '_add_trackable_child',
 '_add_variable_with_custom_getter',
 '_aggregate_exact_metrics',
 '_assert_compile_was_called',
 '_assert_weights_created',
 '_auto_class',
 '_auto_get_config',
 '_auto_track_sub_layers',
 '_autocast',
 '_autographed_call',
 '_autotune_steps_per_execution',
 '_base_model_initialized',
 '_build_input_shape',
 '_call_spec',
 '_callable_losses',
 '_captured_weight_regularizer',
 '_cast_single_input',
 '

In [94]:
vit_model.input_shape

AttributeError: The layer "tf_vi_t_for_image_classification_2" has never been called and thus has no defined input shape. Note that the `input_shape` property is only available for Functional and Sequential models.

In [95]:
from transformers import  TFRobertaModel

In [96]:
roberta = TFRobertaModel.from_pretrained("FacebookAI/roberta-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [97]:
roberta.summary()

Model: "tf_roberta_model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 roberta (TFRobertaMainLaye  multiple                  124645632 
 r)                                                              
                                                                 
Total params: 124645632 (475.49 MB)
Trainable params: 124645632 (475.49 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [98]:
roberta()

ValueError: The first argument to `Layer.call` must always be passed.

In [99]:
hs = vit_model.layers[0](**inputs)['last_hidden_state']
hs.shape

TensorShape([1, 50, 768])

In [100]:
hs

<tf.Tensor: shape=(1, 50, 768), dtype=float32, numpy=
array([[[-0.04642715,  0.07819472,  0.10297327, ..., -0.00410401,
          0.07026549, -0.08159713],
        [-0.01770746, -0.04575405, -0.01447336, ...,  0.00891134,
         -0.03938118,  0.00908835],
        [-0.01938407, -0.04436754, -0.01690471, ...,  0.00544327,
         -0.04128317,  0.00743156],
        ...,
        [ 0.03982006,  0.16118445, -0.205389  , ..., -0.02016526,
          0.01014593, -0.13618861],
        [ 0.03145476,  0.17934817, -0.23624192, ..., -0.03867835,
          0.00920017, -0.15700538],
        [-0.11332527,  0.10698851, -0.01375464, ..., -0.07873282,
          0.02806762, -0.0242648 ]]], dtype=float32)>

In [101]:
res = roberta([],inputs_embeds = hs)

In [102]:
roberta.layers[0]

In [103]:
res.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [104]:
res['last_hidden_state'].shape

TensorShape([1, 50, 768])

In [72]:
# ds = keras.layers.Flatten()(res['last_hidden_state'])
keras.layers.Dense(100)(res['last_hidden_state'])

<tf.Tensor: shape=(1, 197, 100), dtype=float32, numpy=
array([[[ 0.86008716, -0.36986887,  0.0010151 , ..., -0.8080325 ,
          0.47480315, -0.26050305],
        [ 0.70591027,  0.44912875, -0.8911552 , ...,  0.21925709,
         -0.19412321, -0.09676844],
        [ 0.6314004 ,  0.614514  , -0.726822  , ...,  0.34921515,
          0.1237229 , -0.11059649],
        ...,
        [ 0.5519343 ,  0.18524829, -0.6838685 , ...,  0.13580865,
         -0.00276837, -0.28782707],
        [ 0.5669745 ,  0.33112562, -0.79380804, ...,  0.08616413,
          0.14968473, -0.29568923],
        [ 0.53955775,  0.4988504 , -0.83301526, ...,  0.12049282,
          0.20462257, -0.29942286]]], dtype=float32)>

In [73]:
test_word = np.ones((1,3,32,224))

In [78]:
model.vit(test_word)

ValueError: Exception encountered when calling layer 'patch_embeddings' (type TFViTPatchEmbeddings).

Input image size (32*224) doesn't match model (224*224).

Call arguments received by layer 'patch_embeddings' (type TFViTPatchEmbeddings):
  • pixel_values=tf.Tensor(shape=(1, 3, 32, 224), dtype=float32)
  • interpolate_pos_encoding=None
  • training=False

In [152]:
vit_model.summary()

Model: "tf_vi_t_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vit (TFViTMainLayer)        multiple                  88045824  
                                                                 
Total params: 88045824 (335.87 MB)
Trainable params: 88045824 (335.87 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [154]:
vit_model.save('../models/ViT/vit_model.keras')

/home/main/.local/lib/python3.11/site-packages/transformers/generation/tf_utils.py:465: UserWarning: `seed_generator` is deprecated and will be removed in a future version.
  warnings.warn("`seed_generator` is deprecated and will be removed in a future version.", UserWarning)


In [155]:
model = keras.models.load_model('../models/ViT/vit_model.keras')
model.summary()

UnboundLocalError: cannot access local variable 'obj_type' where it is not associated with a value

In [143]:


encoded = vit_model.output
# decoded = roberta([],inputs_embeds =encoded)
logits = keras.layers.Dense(100, activation='softmax')(encoded)
TROCR_model = keras.Model(inputs=vit_model.input, outputs=logits)
TROCR_model.summary()

AttributeError: Layer tf_vi_t_model has no inbound nodes.

In [1]:
import keras_nlp

2024-03-13 11:09:17.123911: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-13 11:09:17.123980: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-13 11:09:17.160022: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-13 11:09:17.236894: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 11:09:18.231204: W tensorflow/compiler/tf2

In [2]:
model = keras_nlp.models.RobertaBackbone.from_preset("roberta_base_en")
model.summary()

2024-03-13 11:09:21.710913: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-13 11:09:21.843419: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-13 11:09:21.843695: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

Model: "roberta_backbone"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ token_ids           │ (None, None)      │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ embeddings          │ (None, None, 768) │ 38,996… │ token_ids[0][0]      │
│ (TokenAndPositionE… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ embeddings_layer_n… │ (None, None, 768) │   1,536 │ embeddings[0][0]     │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ embeddings_dropout  │ (None, None, 768) │       0 │ embeddings_layer_no… │
│ (Dropout)           │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ padding_mask        │ (None, None)      │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transformer_layer_0 │ (None, None, 768) │ 7,087,… │ embeddings_dropout[… │
│ (TransformerEncode… │                   │         │ padding_mask[0][0]   │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transformer_layer_1 │ (None, None, 768) │ 7,087,… │ transformer_layer_0… │
│ (TransformerEncode… │                   │         │ padding_mask[0][0]   │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transformer_layer_2 │ (None, None, 768) │ 7,087,… │ transformer_layer_1… │
│ (TransformerEncode… │                   │         │ padding_mask[0][0]   │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transformer_layer_3 │ (None, None, 768) │ 7,087,… │ transformer_layer_2… │
│ (TransformerEncode… │                   │         │ padding_mask[0][0]   │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transformer_layer_4 │ (None, None, 768) │ 7,087,… │ transformer_layer_3… │
│ (TransformerEncode… │                   │         │ padding_mask[0][0]   │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transformer_layer_5 │ (None, None, 768) │ 7,087,… │ transformer_layer_4… │
│ (TransformerEncode… │                   │         │ padding_mask[0][0]   │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transformer_layer_6 │ (None, None, 768) │ 7,087,… │ transformer_layer_5… │
│ (TransformerEncode… │                   │         │ padding_mask[0][0]   │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transformer_layer_7 │ (None, None, 768) │ 7,087,… │ transformer_layer_6… │
│ (TransformerEncode… │                   │         │ padding_mask[0][0]   │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transformer_layer_8 │ (None, None, 768) │ 7,087,… │ transformer_layer_7… │
│ (TransformerEncode… │                   │         │ padding_mask[0][0]   │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transformer_layer_9 │ (None, None, 768) │ 7,087,… │ transformer_layer_8… │
│ (TransformerEncode… │                   │         │ padding_mask[0][0]   │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transformer_layer_… │ (None, None, 768) │ 7,087,… │ transformer_layer_9… │
│ (TransformerEncode… │                   │         │ padding_mask[0][0]   │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ transformer_layer_… │ (None, None, 768) │ 7,087,… │ transformer_layer_1

 Total params: 124,052,736 (473.22 MB)

 Trainable params: 124,052,736 (473.22 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.layers

[<InputLayer name=token_ids, built=True>,
 <TokenAndPositionEmbedding name=embeddings, built=True>,
 <LayerNormalization name=embeddings_layer_norm, built=True>,
 <Dropout name=embeddings_dropout, built=True>,
 <InputLayer name=padding_mask, built=True>,
 <TransformerEncoder name=transformer_layer_0, built=True>,
 <TransformerEncoder name=transformer_layer_1, built=True>,
 <TransformerEncoder name=transformer_layer_2, built=True>,
 <TransformerEncoder name=transformer_layer_3, built=True>,
 <TransformerEncoder name=transformer_layer_4, built=True>,
 <TransformerEncoder name=transformer_layer_5, built=True>,
 <TransformerEncoder name=transformer_layer_6, built=True>,
 <TransformerEncoder name=transformer_layer_7, built=True>,
 <TransformerEncoder name=transformer_layer_8, built=True>,
 <TransformerEncoder name=transformer_layer_9, built=True>,
 <TransformerEncoder name=transformer_layer_10, built=True>,
 <TransformerEncoder name=transformer_layer_11, built=True>]

In [10]:
import keras

In [19]:
model1 = keras.Sequential()
model1.add(keras.Input(shape=(None, 768)))
for i in range(len(model.layers)):
    if i>4:
        model1.add(model.layers[i])
model1.summary(), model1.layers

Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ transformer_layer_0             │ (None, None, 768)         │  7,087,872 │
│ (TransformerEncoder)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ transformer_layer_1             │ (None, None, 768)         │  7,087,872 │
│ (TransformerEncoder)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ transformer_layer_2             │ (None, None, 768)         │  7,087,872 │
│ (TransformerEncoder)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ transformer_layer_3             │ (None, None, 768)         │  7,087,872 │
│ (TransformerEncoder)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ transformer_layer_4             │ (None, None, 768)         │  7,087,872 │
│ (TransformerEncoder)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ transformer_layer_5             │ (None, None, 768)         │  7,087,872 │
│ (TransformerEncoder)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ transformer_layer_6             │ (None, None, 768)         │  7,087,872 │
│ (TransformerEncoder)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ transformer_layer_7             │ (None, None, 768)         │  7,087,872 │
│ (TransformerEncoder)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ transformer_layer_8             │ (None, None, 768)         │  7,087,872 │
│ (TransformerEncoder)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ transformer_layer_9             │ (None, None, 768)         │  7,087,872 │
│ (TransformerEncoder)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ transformer_layer_10            │ (None, None, 768)         │  7,087,872 │
│ (TransformerEncoder)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ transformer_layer_11            │ (None, None, 768)         │  7,087,872 │
│ (TransformerEncoder)            │                           │            │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 85,054,464 (324.46 MB)

 Trainable params: 85,054,464 (324.46 MB)

 Non-trainable params: 0 (0.00 B)

(None,
 [<TransformerEncoder name=transformer_layer_0, built=True>,
  <TransformerEncoder name=transformer_layer_1, built=True>,
  <TransformerEncoder name=transformer_layer_2, built=True>,
  <TransformerEncoder name=transformer_layer_3, built=True>,
  <TransformerEncoder name=transformer_layer_4, built=True>,
  <TransformerEncoder name=transformer_layer_5, built=True>,
  <TransformerEncoder name=transformer_layer_6, built=True>,
  <TransformerEncoder name=transformer_layer_7, built=True>,
  <TransformerEncoder name=transformer_layer_8, built=True>,
  <TransformerEncoder name=transformer_layer_9, built=True>,
  <TransformerEncoder name=transformer_layer_10, built=True>,
  <TransformerEncoder name=transformer_layer_11, built=True>])

In [34]:
for i in range(len(model1.layers)):
    l = model1.layers[i]
    l1 = model.layers[i+5]
    print(l.weights[0].value - l1.weights[0].value)

tf.Tensor(
[[[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 ...

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]

 [[0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  ...
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]
  [0. 0. 0. ... 0. 0. 0.]]]

In [40]:
(model1.layers[1].weights)

[<KerasVariable shape=(768, 12, 64), dtype=float32, path=transformer_layer_1/self_attention_layer/query/kernel>,
 <KerasVariable shape=(12, 64), dtype=float32, path=transformer_layer_1/self_attention_layer/query/bias>,
 <KerasVariable shape=(768, 12, 64), dtype=float32, path=transformer_layer_1/self_attention_layer/key/kernel>,
 <KerasVariable shape=(12, 64), dtype=float32, path=transformer_layer_1/self_attention_layer/key/bias>,
 <KerasVariable shape=(768, 12, 64), dtype=float32, path=transformer_layer_1/self_attention_layer/value/kernel>,
 <KerasVariable shape=(12, 64), dtype=float32, path=transformer_layer_1/self_attention_layer/value/bias>,
 <KerasVariable shape=(12, 64, 768), dtype=float32, path=transformer_layer_1/self_attention_layer/attention_output/kernel>,
 <KerasVariable shape=(768,), dtype=float32, path=transformer_layer_1/self_attention_layer/attention_output/bias>,
 <KerasVariable shape=(768,), dtype=float32, path=transformer_layer_1/self_attention_layer_norm/gamma>,
 <Ke

In [38]:
import numpy as np
test = np.ones((1,50,768))

In [39]:
model1.predict(test)

2024-03-13 11:30:56.141102: I external/local_xla/xla/service/service.cc:168] XLA service 0xa0d2ae0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-13 11:30:56.141144: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-03-13 11:30:56.205048: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-13 11:30:56.600175: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


I0000 00:00:1710318657.646611    4382 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


array([[[-0.06499024,  0.12883869, -0.0684604 , ..., -0.00282712,
         -0.04725111, -0.02979828],
        [-0.06499024,  0.12883869, -0.0684604 , ..., -0.00282712,
         -0.04725111, -0.02979828],
        [-0.06499024,  0.12883869, -0.0684604 , ..., -0.00282712,
         -0.04725111, -0.02979828],
        ...,
        [-0.06499024,  0.12883869, -0.0684604 , ..., -0.00282712,
         -0.04725111, -0.02979828],
        [-0.06499024,  0.12883869, -0.0684604 , ..., -0.00282712,
         -0.04725111, -0.02979828],
        [-0.06499024,  0.12883869, -0.0684604 , ..., -0.00282712,
         -0.04725111, -0.02979828]]], dtype=float32)

In [41]:
dir(model1.layers[1])

['__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_add_trackable_child',
 '_add_variable_with_custom_getter',
 '_allow_non_tensor_positional_args',
 '_api_export_path',
 '_api_export_symbol_id',
 '_assert_input_compatibility',
 '_auto_config',
 '_build_by_run_for_kwargs',
 '_build_by_run_for_single_pos_arg',
 '_build_shapes_dict',
 '_call_has_mask_arg',
 '_call_has_training_arg',
 '_call_signature',
 '_called',
 '_check_super_called',
 '_checkpoint_dependencies',
 '_clear_losses',
 '_convert_input_args',
 '_copy_trackable_to_cpu',
 '_default_save_signature',
 '_deferred_dependencies',
 '_delete_tracking',
 '_deserializat

In [42]:
l =model1.layers[1] 
l1 = model1.layers[2]

In [44]:
l.weights = l1.weights

ValueError: You cannot add new elements of state (variables or sub-layers) to a layer that is already built. All state must be created in the `__init__()` method or in the `build()` method.

In [81]:
l = keras_nlp.layers.TransformerEncoder.from_config(model1.layers[1].get_config())
# l.build((None, 768))
l.weights

[]

In [73]:
l.set_weights(l1.get_weights())

ValueError: You called `set_weights(weights)` on layer 'transformer_layer_1' with a weight list of length 16, but the layer was expecting 0 weights.

In [74]:
l1.get_weights()

[array([[[-7.78808594e-02, -1.84936523e-02, -3.30200195e-02, ...,
          -6.03942871e-02, -2.22015381e-02,  1.22451782e-03],
         [-4.49523926e-02, -7.23266602e-02,  2.18505859e-02, ...,
           3.89709473e-02,  5.78308105e-02,  5.00488281e-02],
         [ 5.99365234e-02,  2.25067139e-02, -8.50219727e-02, ...,
           1.08642578e-01,  2.05421448e-03,  2.18582153e-03],
         ...,
         [ 5.67626953e-02,  4.89807129e-03,  5.99670410e-03, ...,
           2.73513794e-03, -9.07592773e-02, -4.79431152e-02],
         [-3.17993164e-02,  6.70776367e-02, -5.25512695e-02, ...,
           6.34384155e-03, -9.50927734e-02, -3.26232910e-02],
         [-2.29339600e-02, -7.37304688e-02,  8.08105469e-02, ...,
          -9.87243652e-03, -4.98352051e-02,  3.57360840e-02]],
 
        [[-2.54974365e-02, -8.56323242e-02, -2.86407471e-02, ...,
          -3.13720703e-02,  5.45349121e-02, -3.83605957e-02],
         [ 3.58886719e-02,  1.68762207e-02,  7.18994141e-02, ...,
           9.12475586

In [55]:
l.add_weight()

<KerasVariable shape=(), dtype=float32, path=transformer_layer_1/variable_1>

In [57]:
l.weights.pop(-1)

<KerasVariable shape=(), dtype=float32, path=transformer_layer_1/variable_1>

In [64]:
l.weights 

[<KerasVariable shape=(768, 12, 64), dtype=float32, path=transformer_layer_11/self_attention_layer/query/kernel>,
 <KerasVariable shape=(12, 64), dtype=float32, path=transformer_layer_11/self_attention_layer/query/bias>,
 <KerasVariable shape=(768, 12, 64), dtype=float32, path=transformer_layer_11/self_attention_layer/key/kernel>,
 <KerasVariable shape=(12, 64), dtype=float32, path=transformer_layer_11/self_attention_layer/key/bias>,
 <KerasVariable shape=(768, 12, 64), dtype=float32, path=transformer_layer_11/self_attention_layer/value/kernel>,
 <KerasVariable shape=(12, 64), dtype=float32, path=transformer_layer_11/self_attention_layer/value/bias>,
 <KerasVariable shape=(12, 64, 768), dtype=float32, path=transformer_layer_11/self_attention_layer/attention_output/kernel>,
 <KerasVariable shape=(768,), dtype=float32, path=transformer_layer_11/self_attention_layer/attention_output/bias>,
 <KerasVariable shape=(768,), dtype=float32, path=transformer_layer_11/self_attention_layer_norm/gam

In [62]:
l1.weights

[<KerasVariable shape=(768, 12, 64), dtype=float32, path=transformer_layer_11/self_attention_layer/query/kernel>,
 <KerasVariable shape=(12, 64), dtype=float32, path=transformer_layer_11/self_attention_layer/query/bias>,
 <KerasVariable shape=(768, 12, 64), dtype=float32, path=transformer_layer_11/self_attention_layer/key/kernel>,
 <KerasVariable shape=(12, 64), dtype=float32, path=transformer_layer_11/self_attention_layer/key/bias>,
 <KerasVariable shape=(768, 12, 64), dtype=float32, path=transformer_layer_11/self_attention_layer/value/kernel>,
 <KerasVariable shape=(12, 64), dtype=float32, path=transformer_layer_11/self_attention_layer/value/bias>,
 <KerasVariable shape=(12, 64, 768), dtype=float32, path=transformer_layer_11/self_attention_layer/attention_output/kernel>,
 <KerasVariable shape=(768,), dtype=float32, path=transformer_layer_11/self_attention_layer/attention_output/bias>,
 <KerasVariable shape=(768,), dtype=float32, path=transformer_layer_11/self_attention_layer_norm/gam

AttributeError: 'TransformerEncoder' object has no attribute 'input_shape'

In [66]:
l(np.ones((1,50,768)))

<tf.Tensor: shape=(1, 50, 768), dtype=float32, numpy=
array([[[ 2.2015233 ,  0.761956  , -0.41731355, ...,  0.33482367,
         -1.4790915 , -0.23814008],
        [ 2.2015233 ,  0.761956  , -0.41731355, ...,  0.33482367,
         -1.4790915 , -0.23814008],
        [ 2.2015233 ,  0.761956  , -0.41731355, ...,  0.33482367,
         -1.4790915 , -0.23814008],
        ...,
        [ 2.2015233 ,  0.761956  , -0.41731355, ...,  0.33482367,
         -1.4790915 , -0.23814008],
        [ 2.2015233 ,  0.761956  , -0.41731355, ...,  0.33482367,
         -1.4790915 , -0.23814008],
        [ 2.2015233 ,  0.761956  , -0.41731355, ...,  0.33482367,
         -1.4790915 , -0.23814008]]], dtype=float32)>

In [69]:
len(l.weights), len(l1.weights)

(32, 16)

In [70]:
l.weights[17].value = l.weights[0].value

AttributeError: property 'value' of 'Variable' object has no setter

In [82]:
model1.layers[1].get_config()

{'name': 'transformer_layer_1',
 'trainable': True,
 'dtype': 'float32',
 'intermediate_dim': 3072,
 'num_heads': 12,
 'dropout': 0.1,
 'activation': 'gelu',
 'layer_norm_epsilon': 1e-05,
 'kernel_initializer': {'module': 'keras.src.initializers.random_initializers',
  'class_name': 'TruncatedNormal',
  'config': {'mean': 0.0, 'stddev': 0.02, 'seed': None},
  'registered_name': 'TruncatedNormal'},
 'bias_initializer': {'module': 'keras.src.initializers.constant_initializers',
  'class_name': 'Zeros',
  'config': {},
  'registered_name': 'Zeros'},
 'normalize_first': False}

In [84]:
dec = keras_nlp.layers.TransformerDecoder.from_config(model1.layers[1].get_config())

In [85]:
dec.build((None, 762))

In [86]:
dec.weights 

[<KerasVariable shape=(762, 12, 63), dtype=float32, path=transformer_layer_1/self_attention/query/kernel>,
 <KerasVariable shape=(12, 63), dtype=float32, path=transformer_layer_1/self_attention/query/bias>,
 <KerasVariable shape=(762, 12, 63), dtype=float32, path=transformer_layer_1/self_attention/key/kernel>,
 <KerasVariable shape=(12, 63), dtype=float32, path=transformer_layer_1/self_attention/key/bias>,
 <KerasVariable shape=(762, 12, 63), dtype=float32, path=transformer_layer_1/self_attention/value/kernel>,
 <KerasVariable shape=(12, 63), dtype=float32, path=transformer_layer_1/self_attention/value/bias>,
 <KerasVariable shape=(12, 63, 762), dtype=float32, path=transformer_layer_1/self_attention/attention_output/kernel>,
 <KerasVariable shape=(762,), dtype=float32, path=transformer_layer_1/self_attention/attention_output/bias>,
 <KerasVariable shape=(762,), dtype=float32, path=transformer_layer_1/self_attention_layer_norm/gamma>,
 <KerasVariable shape=(762,), dtype=float32, path=tr

In [87]:
len(dec.weights), len(model1.layers[1].weights)

(16, 16)

In [88]:
dec

<TransformerDecoder name=transformer_layer_1, built=True>

In [93]:
dec1 = keras_nlp.layers.TransformerDecoder( 100, 100)
dec1.build((None, 762), (None, 762))

In [94]:
dec1.weights

[<KerasVariable shape=(762, 100, 7), dtype=float32, path=transformer_decoder_2/self_attention/query/kernel>,
 <KerasVariable shape=(100, 7), dtype=float32, path=transformer_decoder_2/self_attention/query/bias>,
 <KerasVariable shape=(762, 100, 7), dtype=float32, path=transformer_decoder_2/self_attention/key/kernel>,
 <KerasVariable shape=(100, 7), dtype=float32, path=transformer_decoder_2/self_attention/key/bias>,
 <KerasVariable shape=(762, 100, 7), dtype=float32, path=transformer_decoder_2/self_attention/value/kernel>,
 <KerasVariable shape=(100, 7), dtype=float32, path=transformer_decoder_2/self_attention/value/bias>,
 <KerasVariable shape=(100, 7, 762), dtype=float32, path=transformer_decoder_2/self_attention/attention_output/kernel>,
 <KerasVariable shape=(762,), dtype=float32, path=transformer_decoder_2/self_attention/attention_output/bias>,
 <KerasVariable shape=(762,), dtype=float32, path=transformer_decoder_2/self_attention_layer_norm/gamma>,
 <KerasVariable shape=(762,), dtyp

In [97]:
for w in dec1.get_weights():
    print(w.shape)

(762, 100, 7)
(100, 7)
(762, 100, 7)
(100, 7)
(762, 100, 7)
(100, 7)
(100, 7, 762)
(762,)
(762,)
(762,)
(762, 100, 7)
(100, 7)
(762, 100, 7)
(100, 7)
(762, 100, 7)
(100, 7)
(100, 7, 762)
(762,)
(762,)
(762,)
(762, 100)
(100,)
(100, 762)
(762,)
(762,)
(762,)


In [107]:
i=0
for w in dec1.weights:
    print(i,'/'.join(w.path.split('/')[1:]), w.shape)
    i+=1

0 self_attention/query/kernel (762, 100, 7)
1 self_attention/query/bias (100, 7)
2 self_attention/key/kernel (762, 100, 7)
3 self_attention/key/bias (100, 7)
4 self_attention/value/kernel (762, 100, 7)
5 self_attention/value/bias (100, 7)
6 self_attention/attention_output/kernel (100, 7, 762)
7 self_attention/attention_output/bias (762,)
8 self_attention_layer_norm/gamma (762,)
9 self_attention_layer_norm/beta (762,)
10 cross_attention/query/kernel (762, 100, 7)
11 cross_attention/query/bias (100, 7)
12 cross_attention/key/kernel (762, 100, 7)
13 cross_attention/key/bias (100, 7)
14 cross_attention/value/kernel (762, 100, 7)
15 cross_attention/value/bias (100, 7)
16 cross_attention/attention_output/kernel (100, 7, 762)
17 cross_attention/attention_output/bias (762,)
18 cross_attention_layer_norm/gamma (762,)
19 cross_attention_layer_norm/beta (762,)
20 feedforward_intermediate_dense/kernel (762, 100)
21 feedforward_intermediate_dense/bias (100,)
22 feedforward_output_dense/kernel (100,

In [98]:
dir(dec1.weights[0])

['__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__le__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmatmul__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__tf_tensor__',
 '__truediv__',
 '__weakref__',
 '__xor__',
 '_add_trackable_child',
 '_add_variable_with_custom_getter',
 '_checkpoint_dependencies',
 '_convert_to_tensor',
 '_copy_trackable_to_cpu',
 '_deferred_dependencies',
 '_deferred_init

In [104]:
dec1.get_config()

{'name': 'transformer_decoder_2',
 'trainable': True,
 'dtype': 'float32',
 'intermediate_dim': 100,
 'num_heads': 100,
 'dropout': 0,
 'activation': 'relu',
 'layer_norm_epsilon': 1e-05,
 'kernel_initializer': {'module': 'keras.src.initializers.random_initializers',
  'class_name': 'GlorotUniform',
  'config': {'seed': None},
  'registered_name': 'GlorotUniform'},
 'bias_initializer': {'module': 'keras.src.initializers.constant_initializers',
  'class_name': 'Zeros',
  'config': {},
  'registered_name': 'Zeros'},
 'normalize_first': False,
 'decoder_sequence_shape': (None, 762),
 'encoder_sequence_shape': (None, 762)}

In [105]:
model1.layers[1].get_config()

{'name': 'transformer_layer_1',
 'trainable': True,
 'dtype': 'float32',
 'intermediate_dim': 3072,
 'num_heads': 12,
 'dropout': 0.1,
 'activation': 'gelu',
 'layer_norm_epsilon': 1e-05,
 'kernel_initializer': {'module': 'keras.src.initializers.random_initializers',
  'class_name': 'TruncatedNormal',
  'config': {'mean': 0.0, 'stddev': 0.02, 'seed': None},
  'registered_name': 'TruncatedNormal'},
 'bias_initializer': {'module': 'keras.src.initializers.constant_initializers',
  'class_name': 'Zeros',
  'config': {},
  'registered_name': 'Zeros'},
 'normalize_first': False}

In [106]:
dec2 = keras_nlp.layers.TransformerDecoder.from_config(model1.layers[1].get_config())
dec2.build((None, 762), (None, 762))
dec2.weights

[<KerasVariable shape=(762, 12, 63), dtype=float32, path=transformer_layer_1/self_attention/query/kernel>,
 <KerasVariable shape=(12, 63), dtype=float32, path=transformer_layer_1/self_attention/query/bias>,
 <KerasVariable shape=(762, 12, 63), dtype=float32, path=transformer_layer_1/self_attention/key/kernel>,
 <KerasVariable shape=(12, 63), dtype=float32, path=transformer_layer_1/self_attention/key/bias>,
 <KerasVariable shape=(762, 12, 63), dtype=float32, path=transformer_layer_1/self_attention/value/kernel>,
 <KerasVariable shape=(12, 63), dtype=float32, path=transformer_layer_1/self_attention/value/bias>,
 <KerasVariable shape=(12, 63, 762), dtype=float32, path=transformer_layer_1/self_attention/attention_output/kernel>,
 <KerasVariable shape=(762,), dtype=float32, path=transformer_layer_1/self_attention/attention_output/bias>,
 <KerasVariable shape=(762,), dtype=float32, path=transformer_layer_1/self_attention_layer_norm/gamma>,
 <KerasVariable shape=(762,), dtype=float32, path=tr

In [167]:
def convert_encoder_to_decoder(encoder, decoder_input_shape=(None,None, 768)):
    
    decoder = keras_nlp.layers.TransformerDecoder.from_config(encoder.get_config())
    decoder.build(encoder.output.shape, decoder_input_shape)
    
    decoder_weights = []
    
    for dw in decoder.weights:
        dp = '/'.join(dw.path.split('/'))
        print(dw.shape, dp)
        print('cross' in dp)
        for ew in encoder.weights:
            ep = '/'.join(ew.path.split('/'))
            if ep == dp:
                # print(ew.shape, ep)
                decoder_weights.append(ew.value)
                break
        else:
            decoder_weights.append(dw.value)
        #     print('Not found')
        # print('-------------')
    
    # for i in range(len(decoder_weights)):
    #     print(decoder_weights[i].shape, decoder.get_weights()[i].shape)
        

    
    decoder.set_weights(decoder_weights)       
    
    for w in decoder.weights:
        if not 'cross' in w.path:
            w.trainable = False
    
    return decoder

In [168]:
encoder =model1.layers[1]
encoder.output.shape

(None, None, 768)

In [169]:
test_dec = convert_encoder_to_decoder(encoder)

(768, 12, 64) transformer_layer_1/self_attention/query/kernel
False
(12, 64) transformer_layer_1/self_attention/query/bias
False
(768, 12, 64) transformer_layer_1/self_attention/key/kernel
False
(12, 64) transformer_layer_1/self_attention/key/bias
False
(768, 12, 64) transformer_layer_1/self_attention/value/kernel
False
(12, 64) transformer_layer_1/self_attention/value/bias
False
(12, 64, 768) transformer_layer_1/self_attention/attention_output/kernel
False
(768,) transformer_layer_1/self_attention/attention_output/bias
False
(768,) transformer_layer_1/self_attention_layer_norm/gamma
False
(768,) transformer_layer_1/self_attention_layer_norm/beta
False
(768, 12, 64) transformer_layer_1/cross_attention/query/kernel
True
(12, 64) transformer_layer_1/cross_attention/query/bias
True
(768, 12, 64) transformer_layer_1/cross_attention/key/kernel
True
(12, 64) transformer_layer_1/cross_attention/key/bias
True
(768, 12, 64) transformer_layer_1/cross_attention/value/kernel
True
(12, 64) transfor

In [163]:
test_dec.get_weights()

[array([[[-0.02690759,  0.02286571, -0.00200423, ..., -0.02784798,
          -0.01155478,  0.00779839],
         [-0.02644495, -0.0011386 , -0.00867614, ..., -0.03660404,
           0.00416366, -0.02973907],
         [ 0.00899003, -0.01227792,  0.01289736, ...,  0.01615913,
          -0.01047462, -0.02236965],
         ...,
         [-0.00376328,  0.01589137,  0.0152572 , ...,  0.00269613,
          -0.03272806, -0.00936685],
         [ 0.00470315, -0.00966066,  0.01017187, ...,  0.00065113,
          -0.00979351, -0.01906923],
         [-0.01491584, -0.02383434,  0.01677936, ...,  0.02942748,
          -0.0283356 , -0.01442519]],
 
        [[ 0.02535205,  0.02058257,  0.0033271 , ..., -0.02104586,
           0.00791621, -0.03347601],
         [ 0.01010041, -0.03960218,  0.00555957, ..., -0.03945494,
          -0.02972153,  0.01317752],
         [ 0.02232376, -0.03603771,  0.00155826, ..., -0.02211587,
           0.01454037, -0.01563132],
         ...,
         [ 0.02649679, -0.0126270

In [171]:
for w in test_dec.weights:
    print(w.trainable)

False
False
False
False
False
False
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False


In [179]:
test1 = np.ones((1, 17,768))
test2 = np.ones((1, 16,768))


In [181]:
test_dec(test2, test1)

<tf.Tensor: shape=(1, 16, 768), dtype=float32, numpy=
array([[[ 0.29212236,  0.5122221 ,  0.18078876, ..., -0.45685643,
          0.06236216,  0.34362444],
        [ 0.29212236,  0.5122221 ,  0.18078876, ..., -0.45685643,
          0.06236216,  0.34362444],
        [ 0.29212278,  0.51222205,  0.18078874, ..., -0.45685643,
          0.0623626 ,  0.34362447],
        ...,
        [ 0.29212204,  0.51222205,  0.18078887, ..., -0.45685643,
          0.0623621 ,  0.3436246 ],
        [ 0.29212254,  0.5122221 ,  0.18078883, ..., -0.45685667,
          0.06236217,  0.3436246 ],
        [ 0.29212233,  0.5122217 ,  0.18078867, ..., -0.4568563 ,
          0.06236237,  0.34362432]]], dtype=float32)>

In [ ]:
test